In [1]:
import cudf
import pandas as pd
import numpy as np

In [2]:
predict_df = cudf.read_parquet('./data/predicttop10.parquet')
predict_df.head()

,predict,label,recipient,buyAsset-list
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,..."
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]"
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]"
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]"
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]"


In [3]:
predict_df_pd  = predict_df.to_pandas()

In [4]:
predict_df_pd.dtypes

predict          object
label             int64
recipient        object
buyAsset-list    object
dtype: object

In [5]:
def custom_compare(x, y):
    y_list = [int(m) for m in y[1:-1].split()]
    return int(x) in y_list

# Apply the custom function to create a new column 'compare_result'
predict_df_pd['compare_result'] = predict_df_pd.apply(lambda row: custom_compare(row['label'], row['predict']), axis=1)

predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False


In [6]:
predict_df_pd['recipient'].count()

118599

In [7]:
recall = predict_df_pd['compare_result'].sum()/predict_df_pd.shape[0]
recall

0.5943051796389515

# calculate PV

In [8]:
price_table  = cudf.read_parquet(f"./data/price/data.parquet/part_0.parquet")
price_table = price_table.to_pandas()
price_table.head()

,recipient,timestamp-first,buyAsset-list,buyPrice_log_norm-list
0,0x000000000000000000000000000000000000005f,2022-03-12 06:41:35,[478],[0.07]
1,0x00000000000000000000000000000000000000e0,2022-03-17 03:49:58,[478],[17.64]
2,0x00000000000000000000000000000000000000f1,2022-03-13 10:02:30,[478],[0.88]
3,0x0000000000000000000000000000000000000139,2022-03-10 23:21:45,[478],[0.12]
4,0x000000000000000000000000000000000000016e,2022-03-18 00:27:11,[478],[94.19]


In [9]:
def get_PV(col_comapre = None):
    predict_df_pd_captured = predict_df_pd[predict_df_pd[col_comapre]==True][['recipient','label']]
    result_df = pd.merge(predict_df_pd_captured,price_table, on='recipient', how='left')
    result_df['price'] = result_df['buyPrice_log_norm-list'].apply(lambda x: float(x[0]))
    result_df = result_df[result_df['price']<100000000]
    result = result_df['price'].sum()
    print(result)
    return result

In [10]:
print(get_PV(col_comapre = 'compare_result'))

851270271.0799999
851270271.0799999


# load token category encoding

In [12]:
cate_data = cudf.read_parquet(f"./preprocessWorkflow/categories/unique.buyAsset.parquet")
cate_data_pd = cate_data.to_pandas()
cate_data_pd.index

RangeIndex(start=3, stop=1515, step=1)

In [21]:
look_up  = cudf.read_parquet(f"./data/token_lookup/").to_pandas()


# random from list

In [16]:
K=5
def generate_random_integers():
    return list(np.random.randint(3, 1515, size=K))

# Apply the custom function to each row and create a new column 'random_integers'
predict_df_pd[f'random_asset{K}'] = predict_df_pd.apply(lambda row: generate_random_integers(), axis=1)

# Display the resulting DataFrame
predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,random_asset10,compare_result_random10,random_asset5
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1279, 5, 418, 257, 250, 369, 1175, 394, 693, ...",False,"[243, 667, 827, 932, 886]"
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[578, 646, 1439, 795, 187, 1435, 684, 1037, 83...",False,"[801, 1275, 1468, 310, 1212]"
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,"[784, 1071, 948, 515, 753, 1245, 140, 25, 571,...",False,"[894, 465, 1053, 61, 336]"
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,"[388, 784, 772, 861, 364, 1358, 337, 1125, 122...",False,"[699, 1309, 854, 426, 1334]"
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,"[1441, 885, 116, 626, 1357, 1031, 1110, 1451, ...",False,"[274, 735, 98, 757, 657]"


In [17]:
def custom_compare2(x, y):
    return int(x) in y

# Apply the custom function to create a new column 'compare_result'
predict_df_pd[f'compare_result_random{K}'] = predict_df_pd.apply(lambda row: custom_compare2(row['label'], row[f'random_asset{K}']), axis=1)

predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,random_asset10,compare_result_random10,random_asset5,compare_result_random5
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1279, 5, 418, 257, 250, 369, 1175, 394, 693, ...",False,"[243, 667, 827, 932, 886]",False
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[578, 646, 1439, 795, 187, 1435, 684, 1037, 83...",False,"[801, 1275, 1468, 310, 1212]",False
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,"[784, 1071, 948, 515, 753, 1245, 140, 25, 571,...",False,"[894, 465, 1053, 61, 336]",False
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,"[388, 784, 772, 861, 364, 1358, 337, 1125, 122...",False,"[699, 1309, 854, 426, 1334]",False
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,"[1441, 885, 116, 626, 1357, 1031, 1110, 1451, ...",False,"[274, 735, 98, 757, 657]",False


In [18]:
recall = predict_df_pd[f'compare_result_random{K}'].sum()/predict_df_pd.shape[0]
recall

0.0030860293931652036

In [19]:
print(get_PV(col_comapre = f'compare_result_random{K}'))

5537107.67
5537107.67


# random top tokens

In [22]:
look_up = look_up[look_up.token_rank!='Unranked']
look_up.head()

,merge,combined_token_name,combined_token_symbol,token_category,token_rank,risky_flags,bluecheckmark,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,tokenpriceusd
1201,0x5fa54fddf1870c344dbfabb37dfab8700ec0def1,FrogeX,frogex,Decentralized Finance (DeFi),#1514,0,FALSE,"{'combined_token_name': 'Rikkei Finance', 'com...","{'combined_token_name': 'Unicap.finance', 'com...","{'combined_token_name': 'Sukhavati Network', '...","{'combined_token_name': 'Value DeFi', 'combine...","{'combined_token_name': 'GTONCapital', 'combin...","{'combined_token_name': 'McBase', 'combined_to...","{'combined_token_name': 'Cryptoku', 'combined_...","{'combined_token_name': 'Dego Finance', 'combi...","{'combined_token_name': 'Solomon Defi', 'combi...","{'combined_token_name': 'SOBA', 'combined_toke...",3.18E-07
3227,0x5adc961d6ac3f7062d2ea45fefb8d8167d44b190,Dether,DTH,Payments,None,0,TRUE,{'combined_token_name': 'Digital Financial Exc...,"{'combined_token_name': 'Folder Protocol', 'co...","{'combined_token_name': 'Cryptaur', 'combined_...","{'combined_token_name': 'Magic Internet Cash',...",{'combined_token_name': 'Decentralized Crypto ...,"{'combined_token_name': 'Satopay Network', 'co...",{'combined_token_name': 'Black Dragon Society'...,"{'combined_token_name': 'ETH Coin', 'combined_...","{'combined_token_name': 'Cryptopay', 'combined...","{'combined_token_name': 'Cogecoin', 'combined_...",0
3228,0x888666ca69e0f178ded6d75b5726cee99a87d698,ICONOMI,ICN,Crypto Exchanges,None,0,TRUE,"{'combined_token_name': 'CrowdWizToken', 'comb...","{'combined_token_name': 'Robo Inu Finance', 'c...","{'combined_token_name': 'Decentralized ETF', '...","{'combined_token_name': 'InvestFeed', 'combine...","{'combined_token_name': 'Ribbon Finance', 'com...",{'combined_token_name': 'Blackmoon Crypto Toke...,"{'combined_token_name': 'MatrixETF', 'combined...","{'combined_token_name': 'Napoleon X', 'combine...","{'combined_token_name': 'Alpha Bot Calls', 'co...","{'combined_token_name': 'Quantland', 'combined...",0.22
3229,0x7b22938ca841aa392c93dbb7f4c42178e3d65e88,AstroTokens,ASTRO,Staking,None,0,TRUE,"{'combined_token_name': 'Alpha Capital', 'comb...","{'combined_token_name': 'MatrixETF', 'combined...",{'combined_token_name': 'Diversified Staked ET...,"{'combined_token_name': 'BCAP', 'combined_toke...","{'combined_token_name': 'CRYPTO20', 'combined_...","{'combined_token_name': 'Arch Ethereum Web3', ...","{'combined_token_name': 'Decentralized ETF', '...","{'combined_token_name': 'Money Market Index', ...",{'combined_token_name': 'Multi-Chain Capital (...,"{'combined_token_name': 'Bankless BED Index', ...",0.95
3230,0x05f4a42e251f2d52b8ed15e9fedaacfcef1fad27,Zilliqa,ZIL,Blockchain Infrastructure,None,0,TRUE,"{'combined_token_name': nan, 'combined_token_s...","{'combined_token_name': 'SuperEdge', 'combined...","{'combined_token_name': 'ETH Coin', 'combined_...","{'combined_token_name': 'iEthereum', 'combined...",{'combined_token_name': 'Decentralized Crypto ...,"{'combined_token_name': 'NEAR', 'combined_toke...","{'combined_token_name': 'ORACLE', 'combined_to...","{'combined_token_name': 'Chainlink', 'combined...","{'combined_token_name': 'Open Platform', 'comb...","{'combined_token_name': 'IOSToken', 'combined_...",0.02


In [23]:
look_up = look_up[look_up['token_rank'].notna()]

selected_rows = look_up[look_up['token_rank'].str.startswith('#')]

selected_rows.head()

,merge,combined_token_name,combined_token_symbol,token_category,token_rank,risky_flags,bluecheckmark,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,tokenpriceusd
1201,0x5fa54fddf1870c344dbfabb37dfab8700ec0def1,FrogeX,frogex,Decentralized Finance (DeFi),#1514,0,FALSE,"{'combined_token_name': 'Rikkei Finance', 'com...","{'combined_token_name': 'Unicap.finance', 'com...","{'combined_token_name': 'Sukhavati Network', '...","{'combined_token_name': 'Value DeFi', 'combine...","{'combined_token_name': 'GTONCapital', 'combin...","{'combined_token_name': 'McBase', 'combined_to...","{'combined_token_name': 'Cryptoku', 'combined_...","{'combined_token_name': 'Dego Finance', 'combi...","{'combined_token_name': 'Solomon Defi', 'combi...","{'combined_token_name': 'SOBA', 'combined_toke...",3.18E-07
3531,0xf0939011a9bb95c3b791f0cb546377ed2693a574,0.exchange,zero,Crypto Exchanges,#1101,0,None,"{'token_name': 'PowerTrade Fuel', 'token_symbo...","{'token_name': 'Hotbit', 'token_symbol': '[HTB...","{'token_name': 'Pad.Fi', 'token_symbol': '[PAD...","{'token_name': 'Swerve', 'token_symbol': '[SWR...","{'token_name': 'TigerCash', 'token_symbol': '[...","{'token_name': 'XY Finance', 'token_symbol': '...","{'token_name': 'SynchroBitcoin', 'token_symbol...","{'token_name': 'Falconswap', 'token_symbol': '...","{'token_name': 'LuaSwap', 'token_symbol': '[LU...","{'token_name': 'Coinsbit Token', 'token_symbol...",None
3532,0xe41d2489571d322189246dafa5ebde1f4699f498,0x Protocol,zrx,Crypto Exchanges,#85,0,TRUE,"{'token_name': 'Loopring', 'token_symbol': '[L...","{'token_name': 'PancakeSwap', 'token_symbol': ...","{'token_name': 'Golem', 'token_symbol': '[GLM]...","{'token_name': 'MX', 'token_symbol': '[MX]', '...","{'token_name': '1inch', 'token_symbol': '[1INC...","{'token_name': 'Sushi', 'token_symbol': '[SUSH...","{'token_name': 'Osmosis', 'token_symbol': '[OS...","{'token_name': 'Huobi', 'token_symbol': '[HT]'...","{'token_name': 'dYdX', 'token_symbol': '[DYDX]...","{'token_name': 'Bone ShibaSwap', 'token_symbol...",0.29
3533,0xaaca86b876ca011844b5798eca7a67591a9743c8,0x_nodes,bios,Yield Farming,#1326,0,FALSE,"{'token_name': 'ETHA Lend', 'token_symbol': '[...","{'token_name': 'ARTX', 'token_symbol': '[ARTX]...","{'token_name': 'Doki Doki', 'token_symbol': '[...","{'token_name': 'Yield Protocol', 'token_symbol...","{'token_name': 'YVS Finance', 'token_symbol': ...","{'token_name': 'Xiotri', 'token_symbol': '[XIO...","{'token_name': 'Alphr', 'token_symbol': '[ALPH...","{'token_name': 'APY.Finance', 'token_symbol': ...","{'token_name': 'DeHive', 'token_symbol': '[DHV...","{'token_name': '88mph', 'token_symbol': '[MPH]...",0.048724249
3534,0x5a3e6a77ba2f983ec0d371ea3b475f8bc0811ad5,0x0.ai: AI Smart Contract,0x0,Privacy and Cybersecurity,#165,0,FALSE,"{'token_name': 'tomiNet', 'token_symbol': '[TO...","{'token_name': 'Nym', 'token_symbol': '[NYM]',...","{'token_name': 'Orchid Protocol', 'token_symbo...","{'token_name': 'Polymath', 'token_symbol': '[P...","{'token_name': 'Oasis Network', 'token_symbol'...","{'token_name': 'Sentinel Protocol', 'token_sym...","{'token_name': 'Pirate Chain', 'token_symbol':...","{'token_name': 'Hacken Token', 'token_symbol':...","{'token_name': 'Tornado Cash', 'token_symbol':...","{'token_name': 'HOPR', 'token_symbol': '[HOPR]...",0.118972


In [24]:
selected_rows['token_rank'] = selected_rows['token_rank'].apply(lambda x: int(x.replace('#', '')) if isinstance(x, str) else x)

selected_rows.head()

/tmp/ipykernel_17640/1658892853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['token_rank'] = selected_rows['token_rank'].apply(lambda x: int(x.replace('#', '')) if isinstance(x, str) else x)


,merge,combined_token_name,combined_token_symbol,token_category,token_rank,risky_flags,bluecheckmark,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,tokenpriceusd
1201,0x5fa54fddf1870c344dbfabb37dfab8700ec0def1,FrogeX,frogex,Decentralized Finance (DeFi),1514,0,FALSE,"{'combined_token_name': 'Rikkei Finance', 'com...","{'combined_token_name': 'Unicap.finance', 'com...","{'combined_token_name': 'Sukhavati Network', '...","{'combined_token_name': 'Value DeFi', 'combine...","{'combined_token_name': 'GTONCapital', 'combin...","{'combined_token_name': 'McBase', 'combined_to...","{'combined_token_name': 'Cryptoku', 'combined_...","{'combined_token_name': 'Dego Finance', 'combi...","{'combined_token_name': 'Solomon Defi', 'combi...","{'combined_token_name': 'SOBA', 'combined_toke...",3.18E-07
3531,0xf0939011a9bb95c3b791f0cb546377ed2693a574,0.exchange,zero,Crypto Exchanges,1101,0,None,"{'token_name': 'PowerTrade Fuel', 'token_symbo...","{'token_name': 'Hotbit', 'token_symbol': '[HTB...","{'token_name': 'Pad.Fi', 'token_symbol': '[PAD...","{'token_name': 'Swerve', 'token_symbol': '[SWR...","{'token_name': 'TigerCash', 'token_symbol': '[...","{'token_name': 'XY Finance', 'token_symbol': '...","{'token_name': 'SynchroBitcoin', 'token_symbol...","{'token_name': 'Falconswap', 'token_symbol': '...","{'token_name': 'LuaSwap', 'token_symbol': '[LU...","{'token_name': 'Coinsbit Token', 'token_symbol...",None
3532,0xe41d2489571d322189246dafa5ebde1f4699f498,0x Protocol,zrx,Crypto Exchanges,85,0,TRUE,"{'token_name': 'Loopring', 'token_symbol': '[L...","{'token_name': 'PancakeSwap', 'token_symbol': ...","{'token_name': 'Golem', 'token_symbol': '[GLM]...","{'token_name': 'MX', 'token_symbol': '[MX]', '...","{'token_name': '1inch', 'token_symbol': '[1INC...","{'token_name': 'Sushi', 'token_symbol': '[SUSH...","{'token_name': 'Osmosis', 'token_symbol': '[OS...","{'token_name': 'Huobi', 'token_symbol': '[HT]'...","{'token_name': 'dYdX', 'token_symbol': '[DYDX]...","{'token_name': 'Bone ShibaSwap', 'token_symbol...",0.29
3533,0xaaca86b876ca011844b5798eca7a67591a9743c8,0x_nodes,bios,Yield Farming,1326,0,FALSE,"{'token_name': 'ETHA Lend', 'token_symbol': '[...","{'token_name': 'ARTX', 'token_symbol': '[ARTX]...","{'token_name': 'Doki Doki', 'token_symbol': '[...","{'token_name': 'Yield Protocol', 'token_symbol...","{'token_name': 'YVS Finance', 'token_symbol': ...","{'token_name': 'Xiotri', 'token_symbol': '[XIO...","{'token_name': 'Alphr', 'token_symbol': '[ALPH...","{'token_name': 'APY.Finance', 'token_symbol': ...","{'token_name': 'DeHive', 'token_symbol': '[DHV...","{'token_name': '88mph', 'token_symbol': '[MPH]...",0.048724249
3534,0x5a3e6a77ba2f983ec0d371ea3b475f8bc0811ad5,0x0.ai: AI Smart Contract,0x0,Privacy and Cybersecurity,165,0,FALSE,"{'token_name': 'tomiNet', 'token_symbol': '[TO...","{'token_name': 'Nym', 'token_symbol': '[NYM]',...","{'token_name': 'Orchid Protocol', 'token_symbo...","{'token_name': 'Polymath', 'token_symbol': '[P...","{'token_name': 'Oasis Network', 'token_symbol'...","{'token_name': 'Sentinel Protocol', 'token_sym...","{'token_name': 'Pirate Chain', 'token_symbol':...","{'token_name': 'Hacken Token', 'token_symbol':...","{'token_name': 'Tornado Cash', 'token_symbol':...","{'token_name': 'HOPR', 'token_symbol': '[HOPR]...",0.118972


In [25]:
selected_rows_sorted = selected_rows.sort_values(by='token_rank')
selected_rows_sorted.head()

,merge,combined_token_name,combined_token_symbol,token_category,token_rank,risky_flags,bluecheckmark,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,tokenpriceusd
3987,0x0000000000000000000000000000000000000eth,Ethereum,ETH,Blockchain Networks,1,0,None,"{'token_name': 'Polygon', 'token_symbol': '[MA...","{'token_name': 'Fantom', 'token_symbol': '[FTM...","{'token_name': 'Wrapped Flow', 'token_symbol':...","{'token_name': 'Celer Network', 'token_symbol'...","{'token_name': 'OMG Network', 'token_symbol': ...","{'token_name': 'Metis', 'token_symbol': '[METI...","{'token_name': 'Tokamak Network', 'token_symbo...","{'token_name': 'Aurora', 'token_symbol': '[AUR...","{'token_name': 'Hermez Network', 'token_symbol...","{'token_name': 'Boba Network', 'token_symbol':...",None
4808,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether,usdt,Stablecoins,2,1,TRUE,"{'token_name': 'USDC', 'token_symbol': '[USDC]...","{'token_name': 'Dai', 'token_symbol': '[DAI]',...","{'token_name': 'TrueUSD', 'token_symbol': '[TU...","{'token_name': 'BUSD', 'token_symbol': '[BUSD]...","{'token_name': 'USDD', 'token_symbol': '[USDD]...","{'token_name': 'Frax', 'token_symbol': '[FRAX]...","{'token_name': 'First Digital USD', 'token_sym...","{'token_name': 'Tether Gold', 'token_symbol': ...","{'token_name': 'PAX Gold', 'token_symbol': '[P...","{'token_name': 'Pax Dollar', 'token_symbol': '...",1
4899,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,usdc,Stablecoins,4,0,TRUE,"{'token_name': 'Dai', 'token_symbol': '[DAI]',...","{'token_name': 'TrueUSD', 'token_symbol': '[TU...","{'token_name': 'BUSD', 'token_symbol': '[BUSD]...","{'token_name': 'USDD', 'token_symbol': '[USDD]...","{'token_name': 'Frax', 'token_symbol': '[FRAX]...","{'token_name': 'First Digital USD', 'token_sym...","{'token_name': 'Tether Gold', 'token_symbol': ...","{'token_name': 'PAX Gold', 'token_symbol': '[P...","{'token_name': 'Pax Dollar', 'token_symbol': '...","{'token_name': 'Liquity USD', 'token_symbol': ...",1
4283,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,Lido Staked Ether,steth,Staking,5,0,TRUE,"{'token_name': 'Lido DAO', 'token_symbol': '[L...","{'token_name': 'HEX', 'token_symbol': '[HEX]',...","{'token_name': 'Wrapped Beacon ETH', 'token_sy...","{'token_name': 'Rocket Pool', 'token_symbol': ...","{'token_name': 'cETH', 'token_symbol': '[CETH]...","{'token_name': 'Blox', 'token_symbol': '[CDT]'...","{'token_name': 'Ankr Staked ETH', 'token_symbo...","{'token_name': 'Gomining Token', 'token_symbol...","{'token_name': 'Marinade Staked SOL', 'token_s...","{'token_name': 'Stader', 'token_symbol': '[SD]...","1,796.46"
4838,0x582d872a1b094fc48f5de31d3b73f2d9be47def1,Toncoin,ton,Payments,6,0,TRUE,"{'token_name': 'Terra Classic', 'token_symbol'...","{'token_name': 'NEXO', 'token_symbol': '[NEXO]...","{'token_name': 'SwissBorg', 'token_symbol': '[...","{'token_name': 'JasmyCoin', 'token_symbol': '[...","{'token_name': 'Fasttoken', 'token_symbol': '[...","{'token_name': 'PayPal USD', 'token_symbol': '...","{'token_name': 'SXP', 'token_symbol': '[SXP]',...","{'token_name': 'Telcoin', 'token_symbol': '[TE...","{'token_name': 'AirTor Protocol', 'token_symbo...","{'token_name': 'Pundi X', 'token_symbol': '[PU...",2.21


In [28]:
top_Assets = selected_rows_sorted[selected_rows_sorted.token_rank<100]['merge']

In [29]:
mapping = cate_data_pd.reset_index()[['index','buyAsset']]
index2Asset_dict = dict(zip(mapping['index'], mapping['buyAsset']))
asset2Index_dict = dict(zip(mapping['buyAsset'], mapping['index']))


In [30]:
topAssets_encoded = [ asset2Index_dict[x]  if x in asset2Index_dict.keys() else -1 for x in top_Assets.values]

In [31]:
filtered_list = list(filter(lambda x: x != -1, topAssets_encoded))
len(filtered_list)

68

In [37]:
K=5
def generate_random_top():
    import random
    return random.sample(filtered_list, K)
# Apply the custom function to each row and create a new column 'random_integers'
predict_df_pd[f'random_asset_top{K}'] = predict_df_pd.apply(lambda row: generate_random_top(), axis=1)

# Display the resulting DataFrame
predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,random_asset10,compare_result_random10,random_asset5,compare_result_random5,random_asset_top10,compare_result_top10,random_asset_top5
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1279, 5, 418, 257, 250, 369, 1175, 394, 693, ...",False,"[243, 667, 827, 932, 886]",False,"[84, 100, 81, 869, 235, 283, 622, 21, 361, 193]",False,"[987, 8, 100, 361, 32]"
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[578, 646, 1439, 795, 187, 1435, 684, 1037, 83...",False,"[801, 1275, 1468, 310, 1212]",False,"[105, 5, 11, 1180, 213, 23, 193, 721, 125, 788]",False,"[198, 174, 1307, 149, 523]"
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,"[784, 1071, 948, 515, 753, 1245, 140, 25, 571,...",False,"[894, 465, 1053, 61, 336]",False,"[3, 218, 235, 927, 404, 63, 632, 5, 8, 361]",True,"[155, 38, 1176, 670, 65]"
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,"[388, 784, 772, 861, 364, 1358, 337, 1125, 122...",False,"[699, 1309, 854, 426, 1334]",False,"[546, 23, 291, 869, 49, 523, 48, 1180, 183, 6]",False,"[6, 166, 22, 523, 155]"
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,"[1441, 885, 116, 626, 1357, 1031, 1110, 1451, ...",False,"[274, 735, 98, 757, 657]",False,"[404, 5, 140, 7, 203, 315, 22, 721, 745, 670]",True,"[23, 41, 174, 6, 1176]"


In [38]:
def custom_compare2(x, y):
    return int(x) in y

# Apply the custom function to create a new column 'compare_result'
predict_df_pd[f'compare_result_top{K}'] = predict_df_pd.apply(lambda row: custom_compare2(row['label'], row[f'random_asset_top{K}']), axis=1)

predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,random_asset10,compare_result_random10,random_asset5,compare_result_random5,random_asset_top10,compare_result_top10,random_asset_top5,compare_result_top5
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1279, 5, 418, 257, 250, 369, 1175, 394, 693, ...",False,"[243, 667, 827, 932, 886]",False,"[84, 100, 81, 869, 235, 283, 622, 21, 361, 193]",False,"[987, 8, 100, 361, 32]",False
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[578, 646, 1439, 795, 187, 1435, 684, 1037, 83...",False,"[801, 1275, 1468, 310, 1212]",False,"[105, 5, 11, 1180, 213, 23, 193, 721, 125, 788]",False,"[198, 174, 1307, 149, 523]",False
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,"[784, 1071, 948, 515, 753, 1245, 140, 25, 571,...",False,"[894, 465, 1053, 61, 336]",False,"[3, 218, 235, 927, 404, 63, 632, 5, 8, 361]",True,"[155, 38, 1176, 670, 65]",False
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,"[388, 784, 772, 861, 364, 1358, 337, 1125, 122...",False,"[699, 1309, 854, 426, 1334]",False,"[546, 23, 291, 869, 49, 523, 48, 1180, 183, 6]",False,"[6, 166, 22, 523, 155]",False
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,"[1441, 885, 116, 626, 1357, 1031, 1110, 1451, ...",False,"[274, 735, 98, 757, 657]",False,"[404, 5, 140, 7, 203, 315, 22, 721, 745, 670]",True,"[23, 41, 174, 6, 1176]",False


In [40]:
recall = predict_df_pd[f'compare_result_top{K}'].sum()/predict_df_pd.shape[0]
print(recall)

0.03387043735613285


In [41]:
print(get_PV(col_comapre = f'compare_result_top{K}'))

68234203.97
68234203.97


# random from past list


In [67]:
def past_tokens(x):
    return x[:-1]

# Apply the custom function to create a new column 'compare_result'
predict_df_pd['pastTokens'] = predict_df_pd.apply(lambda row: past_tokens(row['buyAsset-list']), axis=1)

predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,pastTokens
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,..."
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2]"
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,[7]
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,[3]
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,[4]


In [75]:
K=5
def generate_random_from_past(x):
    import random
    return random.choices(x, k=K)
predict_df_pd[f'random_asset_top{K}'] = predict_df_pd.apply(lambda row: generate_random_from_past(row['pastTokens']), axis=1)

# Display the resulting DataFrame
predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,pastTokens,random_asset_top10,compare_result_past10,random_asset_top5
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...","[941, 293, 714, 540, 17, 388, 17, 714, 540, 503]",False,"[17, 211, 1414, 293, 503]"
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2]","[4, 5, 5, 113, 5, 4, 4, 5, 3, 4]",True,"[5, 4, 3, 5, 4]"
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,[7],"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]",False,"[7, 7, 7, 7, 7]"
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,[3],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",False,"[3, 3, 3, 3, 3]"
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,[4],"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",False,"[4, 4, 4, 4, 4]"


In [76]:
def custom_compare3(x, y):
    return int(x) in y

# Apply the custom function to create a new column 'compare_result'
predict_df_pd[f'compare_result_past{K}'] = predict_df_pd.apply(lambda row: custom_compare3(row['label'], row[f'random_asset_top{K}']), axis=1)

predict_df_pd.head()

,predict,label,recipient,buyAsset-list,compare_result,pastTokens,random_asset_top10,compare_result_past10,random_asset_top5,compare_result_past5
0,[ 5 4 3 37 14 182 25 18 30 24],4,0x000000000004d7463d0f9c77383600bc82d612f5,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...",True,"[1476, 1017, 714, 503, 1414, 540, 24, 57, 388,...","[941, 293, 714, 540, 17, 388, 17, 714, 540, 503]",False,"[17, 211, 1414, 293, 503]",False
1,[ 4 3 5 15 31 53 16 8 13 34],4,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2, 4]",True,"[4, 5, 113, 4, 5, 4, 3, 4, 5, 4, 2]","[4, 5, 5, 113, 5, 4, 4, 5, 3, 4]",True,"[5, 4, 3, 5, 4]",True
2,[ 3 5 7 4 9 12 8 21 28 11],3,0x00000000005dbcb0d0513fcda746382fe8a53468,"[7, 3]",True,[7],"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]",False,"[7, 7, 7, 7, 7]",False
3,[ 3 4 5 8 7 9 11 6 12 21],252,0x000000000088e0120f9e6652cc058aec07564f69,"[3, 252]",False,[3],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",False,"[3, 3, 3, 3, 3]",False
4,[ 4 3 5 8 16 13 11 14 15 31],7,0x0000000000e002b4e3009de1c7614ae5e1facf7a,"[4, 7]",False,[4],"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",False,"[4, 4, 4, 4, 4]",False


In [77]:
recall = predict_df_pd[f'compare_result_past{K}'].sum()/predict_df_pd.shape[0]
print(recall)

0.2571691160971003


In [78]:
print(get_PV(col_comapre = f'compare_result_past{K}'))

372371005.25
372371005.25


In [71]:
look_up = look_up[['merge','top_1','top_2','top_3','top_4','top_5','top_6','top_7','top_8','top_9','top_10']]

In [72]:
look_up.head()

,merge,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10
0,0xa50709b10c328772ea24a44b13c7e23c75561ecb,"{'combined_token_name': 'Titanium22', 'combine...","{'combined_token_name': 'Sunrise', 'combined_t...","{'combined_token_name': 'TempleDAO', 'combined...","{'combined_token_name': 'GameStop.Finance', 'c...","{'combined_token_name': 'Rake Casino', 'combin...","{'combined_token_name': 'Tail', 'combined_toke...","{'combined_token_name': 'Wall Street Memes', '...","{'combined_token_name': 'BuyMORE', 'combined_t...","{'combined_token_name': 'Big Eyes', 'combined_...","{'combined_token_name': 'SafeBlast', 'combined..."
1,0xe4efdd2eb216a4620cfa55c5cc67bd09dc64ff24,"{'combined_token_name': 'Cash Token', 'combine...","{'combined_token_name': 'Neural Protocol', 'co...","{'combined_token_name': 'Bitcoin ETF', 'combin...","{'combined_token_name': 'Magic Internet Cash',...","{'combined_token_name': 'ETH Coin', 'combined_...","{'combined_token_name': 'TON Community', 'comb...",{'combined_token_name': 'Decentralized Crypto ...,{'combined_token_name': 'Digital Financial Exc...,{'combined_token_name': 'The Tokenized Bitcoin...,"{'combined_token_name': 'YOLOCash', 'combined_..."
2,0x3d2b66bc4f9d6388bd2d97b95b565be1686aefb3,80774,$LAMBO,lambo,"{'combined_token_name': 'Dream', 'combined_tok...","{'combined_token_name': 'WEN Token', 'combined...","{'combined_token_name': 'Baby Shiba Coin', 'co...","{'combined_token_name': 'Wojak 2.69', 'combine...","{'combined_token_name': 'Bull Market', 'combin...","{'combined_token_name': 'Patrick', 'combined_t...","{'combined_token_name': 'MicroPepe', 'combined..."
3,0x48af7b1c9dac8871c064f62fcec0d9d6f7c269f5,"{'combined_token_name': 'Alpha Intelligence', ...","{'combined_token_name': 'Stella', 'combined_to...","{'combined_token_name': 'Alpha Coin', 'combine...","{'combined_token_name': 'Bot Compiler', 'combi...","{'combined_token_name': 'Alpha Token', 'combin...","{'combined_token_name': 'Alpha Shards', 'combi...","{'combined_token_name': 'Alpha Gardeners', 'co...",{'combined_token_name': 'Q DAO Governance toke...,"{'combined_token_name': 'Hackerlabs DAO', 'com...","{'combined_token_name': 'Alpha Impact', 'combi..."
4,0x4594cffbfc09bc5e7ecf1c2e1c1e24f0f7d29036,"{'combined_token_name': 'Evanesco Network', 'c...","{'combined_token_name': 'Secret', 'combined_to...","{'combined_token_name': 'Oasis Network', 'comb...","{'combined_token_name': 'BlackHat', 'combined_...","{'combined_token_name': 'LTO Network', 'combin...","{'combined_token_name': 'SCRIV', 'combined_tok...",{'combined_token_name': 'Public Index Network'...,"{'combined_token_name': 'ZkTsunami', 'combined...","{'combined_token_name': 'No One', 'combined_to...","{'combined_token_name': 'Sentinel [OLD]', 'com..."


In [75]:
def encode(x):
    if x in asset2Index_dict.keys():
        return int(asset2Index_dict[x])
    else:
        return None

# Apply the custom function to create a new column 'compare_result'
look_up['merge_encoded'] = look_up.apply(lambda row: encode(row['merge']), axis=1)

look_up.head()

,merge,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,merge_encoded
0,0xa50709b10c328772ea24a44b13c7e23c75561ecb,"{'combined_token_name': 'Titanium22', 'combine...","{'combined_token_name': 'Sunrise', 'combined_t...","{'combined_token_name': 'TempleDAO', 'combined...","{'combined_token_name': 'GameStop.Finance', 'c...","{'combined_token_name': 'Rake Casino', 'combin...","{'combined_token_name': 'Tail', 'combined_toke...","{'combined_token_name': 'Wall Street Memes', '...","{'combined_token_name': 'BuyMORE', 'combined_t...","{'combined_token_name': 'Big Eyes', 'combined_...","{'combined_token_name': 'SafeBlast', 'combined...",NaN
1,0xe4efdd2eb216a4620cfa55c5cc67bd09dc64ff24,"{'combined_token_name': 'Cash Token', 'combine...","{'combined_token_name': 'Neural Protocol', 'co...","{'combined_token_name': 'Bitcoin ETF', 'combin...","{'combined_token_name': 'Magic Internet Cash',...","{'combined_token_name': 'ETH Coin', 'combined_...","{'combined_token_name': 'TON Community', 'comb...",{'combined_token_name': 'Decentralized Crypto ...,{'combined_token_name': 'Digital Financial Exc...,{'combined_token_name': 'The Tokenized Bitcoin...,"{'combined_token_name': 'YOLOCash', 'combined_...",NaN
2,0x3d2b66bc4f9d6388bd2d97b95b565be1686aefb3,80774,$LAMBO,lambo,"{'combined_token_name': 'Dream', 'combined_tok...","{'combined_token_name': 'WEN Token', 'combined...","{'combined_token_name': 'Baby Shiba Coin', 'co...","{'combined_token_name': 'Wojak 2.69', 'combine...","{'combined_token_name': 'Bull Market', 'combin...","{'combined_token_name': 'Patrick', 'combined_t...","{'combined_token_name': 'MicroPepe', 'combined...",NaN
3,0x48af7b1c9dac8871c064f62fcec0d9d6f7c269f5,"{'combined_token_name': 'Alpha Intelligence', ...","{'combined_token_name': 'Stella', 'combined_to...","{'combined_token_name': 'Alpha Coin', 'combine...","{'combined_token_name': 'Bot Compiler', 'combi...","{'combined_token_name': 'Alpha Token', 'combin...","{'combined_token_name': 'Alpha Shards', 'combi...","{'combined_token_name': 'Alpha Gardeners', 'co...",{'combined_token_name': 'Q DAO Governance toke...,"{'combined_token_name': 'Hackerlabs DAO', 'com...","{'combined_token_name': 'Alpha Impact', 'combi...",560.0
4,0x4594cffbfc09bc5e7ecf1c2e1c1e24f0f7d29036,"{'combined_token_name': 'Evanesco Network', 'c...","{'combined_token_name': 'Secret', 'combined_to...","{'combined_token_name': 'Oasis Network', 'comb...","{'combined_token_name': 'BlackHat', 'combined_...","{'combined_token_name': 'LTO Network', 'combin...","{'combined_token_name': 'SCRIV', 'combined_tok...",{'combined_token_name': 'Public Index Network'...,"{'combined_token_name': 'ZkTsunami', 'combined...","{'combined_token_name': 'No One', 'combined_to...","{'combined_token_name': 'Sentinel [OLD]', 'com...",NaN


In [79]:
look_up_select = look_up[look_up.merge_encoded > 0]
look_up_select.head()

,merge,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,merge_encoded
3,0x48af7b1c9dac8871c064f62fcec0d9d6f7c269f5,"{'combined_token_name': 'Alpha Intelligence', ...","{'combined_token_name': 'Stella', 'combined_to...","{'combined_token_name': 'Alpha Coin', 'combine...","{'combined_token_name': 'Bot Compiler', 'combi...","{'combined_token_name': 'Alpha Token', 'combin...","{'combined_token_name': 'Alpha Shards', 'combi...","{'combined_token_name': 'Alpha Gardeners', 'co...",{'combined_token_name': 'Q DAO Governance toke...,"{'combined_token_name': 'Hackerlabs DAO', 'com...","{'combined_token_name': 'Alpha Impact', 'combi...",560.0
15,0x525794473f7ab5715c81d06d10f52d11cc052804,{'combined_token_name': 'Speed Mining Service'...,"{'combined_token_name': 'Ormeus Coin', 'combin...",{'combined_token_name': 'Speed Mining Service'...,"{'combined_token_name': 'Lumerin', 'combined_t...",{'combined_token_name': 'Vehicle Mining System...,"{'combined_token_name': 'ASIC Token', 'combine...",{'combined_token_name': 'Ethereum Gold Mining ...,"{'combined_token_name': 'Quantstamp', 'combine...","{'combined_token_name': 'Eminer', 'combined_to...","{'combined_token_name': 'Chainlink', 'combined...",1408.0
31,0x3f5294df68f871241c4b18fcf78ebd8ac18ab654,"{'combined_token_name': 'Smart Game Finance', ...","{'combined_token_name': 'GameStop.Finance', 'c...","{'combined_token_name': 'X World Games', 'comb...","{'combined_token_name': 'G*', 'combined_token_...","{'combined_token_name': 'BlockchainSpace', 'co...","{'combined_token_name': 'Galaxy Villains', 'co...","{'combined_token_name': 'Versailles Heroes', '...","{'combined_token_name': 'MeGods', 'combined_to...","{'combined_token_name': 'RUGAME', 'combined_to...","{'combined_token_name': 'MetaRuffy (MR)', 'com...",818.0
37,0x5ba19d656b65f1684cfea4af428c23b9f3628f97,"{'combined_token_name': 'Arch Ethereum Web3', ...","{'combined_token_name': 'Ankr Network', 'combi...","{'combined_token_name': 'Alpha Token', 'combin...","{'combined_token_name': 'Rangers Protocol', 'c...","{'combined_token_name': 'Changer', 'combined_t...","{'combined_token_name': 'Connectome', 'combine...","{'combined_token_name': '00 Token', 'combined_...","{'combined_token_name': 'UnitedCrowd', 'combin...","{'combined_token_name': '0.exchange', 'combine...","{'combined_token_name': 'FINU', 'combined_toke...",575.0
68,0xfc1e690f61efd961294b3e1ce3313fbd8aa4f85d,"{'combined_token_name': 'Aave DAI', 'combined_...","{'combined_token_name': 'Aave AMM UniDAIUSDC',...","{'combined_token_name': 'Aave AMM DAI', 'combi...","{'combined_token_name': 'Aave WETH', 'combined...","{'combined_token_name': 'Aave CRV', 'combined_...","{'combined_token_name': 'Aave TUSD v1', 'combi...","{'combined_token_name': 'Aave AAVE', 'combined...","{'combined_token_name': 'Aave RAI', 'combined_...",{'combined_token_name': 'Aave AMM UniAAVEWETH'...,"{'combined_token_name': 'Aave AMM BptBALWETH',...",1343.0


/tmp/ipykernel_11613/63936846.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  look_up_select['top_1_encode'] = look_up_select.apply(lambda row: encode_top(row['top_3']), axis=1)
